In [ ]:
import os
import cv2
import shutil
import PIL
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [ ]:
BASE_DIR = '../'
MODEL_DIR = BASE_DIR+'model/'
TEST_DIR = BASE_DIR+'test/'
IMAGES_DIR = TEST_DIR+'images/src-images/'
CROPPED_DIR = TEST_DIR+'images/test-images/cropped-faces/'
PREDICTIONS_DIR = TEST_DIR+'predictions/'
IMG_HEIGHT = 256
IMG_WIDTH = 256

In [ ]:
def get_face(img, x, y, w, h) -> Image.Image: 
    crop_img = img[y:y+h, x:x+w]
    rgb_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_img)
    return pil_img

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for filename in tqdm(os.listdir(IMAGES_DIR)):
    if os.path.isdir(IMAGES_DIR+filename) or filename == '.DS_Store':
        print(f'{bcolors.WARNING}[INFO]: Skipping {filename}: is a directory ...{bcolors.RESET}')
        continue
    img = cv2.imread(IMAGES_DIR+filename)
    try: 
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except:
        print(f'{bcolors.FAIL}[ERROR]: Unable to call cv2.cvtColor for {filename}...{bcolors.RESET}')
        continue
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        faces = img[y:y + h, x:x + w]
        new_filename = CROPPED_DIR+filename+'-face.jpeg'
        if os.path.isfile(new_filename):
            print(f'{bcolors.WARNING}[INFO]: Avoiding saving {filename}: already in the destination folder ...{bcolors.RESET}')
            continue
        cv2.imwrite(new_filename, faces)